In [1]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#### Constants

In [2]:
first_cols = ['cardNum', 'boarding_datetime','gps_datetime','route','busCode','stopPointId']
boarding_key_cols = ['cardNum','boarding_datetime']
max_match_diff = 1500

#### Read BUSTE versions

In [3]:
buste_v1 = pd.read_csv('/local/tarciso/masters/data/bus_trips/buste-v3a/may_jun_jul/single-csvs/2017_05_09_veiculos.csv', dtype = {'route': str}, na_values='-', low_memory=False)

In [4]:
buste_v2 = pd.read_csv('/local/tarciso/masters/data/bus_trips/test/part-00000-buste-latest', dtype = {'route': str}, na_values='-')

In [4]:
buste_v1.head()

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,gpsLon,distanceToShapePoint,timestamp,stopPointId,problem,birthdate,cardTimestamp,lineName,cardNum,gender
0,500,2.0,4130,6464140,-25.438837,-49.268129,9735.966,GE718,NaN,-25.438738,-49.268180,12.105323,06:53:44,25515,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN
1,500,2.0,4130,6464167,-25.433790,-49.270262,10543.422,GE718,NaN,-25.433916,-49.270425,21.556105,06:58:25,27560,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN
2,500,5.0,4130,6463793,-25.516848,-49.230057,0.000,GE718,NaN,-25.516885,-49.229913,15.014615,07:22:26,27537,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN
3,500,1.0,4130,6463875,-25.501550,-49.237590,2129.862,GE718,NaN,NaN,NaN,NaN,07:26:47,27472,BETWEEN,NaN,NaN,NaN,NaN,NaN
4,500,1.0,4130,6463945,-25.481825,-49.246977,4513.260,GE718,NaN,NaN,NaN,NaN,07:28:04,27551,BETWEEN,NaN,NaN,NaN,NaN,NaN


In [5]:
buste_v2.head()

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,distanceToShapePoint,timestamp,stopPointId,problem,birthdate,cardTimestamp,lineName,cardNum,gender,date
0,372,4.0,1891,6136674,-25.418008,-49.201094,985.183,CC170,NaN,NaN,...,NaN,12:00:03,30778,BETWEEN,NaN,NaN,NaN,NaN,NaN,2017_05_09
1,372,4.0,1891,6136668,-25.416008,-49.202063,743.173,CC170,NaN,NaN,...,NaN,12:00:52,30767,BETWEEN,NaN,NaN,NaN,NaN,NaN,2017_05_09
2,372,4.0,1891,6136658,-25.413175,-49.203970,326.247,CC170,NaN,NaN,...,NaN,12:02:16,30760,BETWEEN,NaN,NaN,NaN,NaN,NaN,2017_05_09
3,372,4.0,1891,6136644,-25.412986,-49.205264,0.000,CC170,NaN,-25.413055,...,11.568364,12:03:22,26210,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN,2017_05_09
4,372,5.0,1891,6136658,-25.413175,-49.203970,326.247,CC170,NaN,-25.413151,...,24.480747,12:04:49,30760,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN,2017_05_09


In [6]:
print "length: v1: {}, v2: {}".format(len(buste_v1),len(buste_v2))

length: v1: 1303692, v2: 642639


#### Add date and datetime to gps dataframe

In [5]:
buste_v1['date'] = '2017_05_09'
buste_v1['gps_datetime'] = pd.to_datetime(buste_v1['date'] + ' ' + buste_v1['timestamp'],format='%Y_%m_%d %H:%M:%S')
buste_v1['boarding_datetime'] = pd.to_datetime(buste_v1['date'] + ' ' + buste_v1['cardTimestamp'],format='%Y_%m_%d %H:%M:%S')
buste_v1['date'] = pd.to_datetime(buste_v1['date'], format='%Y_%m_%d') 
buste_v1 = buste_v1.reset_index(drop=True)

In [29]:
print "Num unique boardings v1: {}".format(len(buste_v1.drop_duplicates(boarding_key_cols)))

Num unique boardings v1: 152707


In [8]:
buste_v1[['date','gps_datetime','boarding_datetime']].sort_values('gps_datetime')

,date,gps_datetime,boarding_datetime
591312,2017-05-09,2017-05-09 00:00:20,NaT
591316,2017-05-09,2017-05-09 00:04:17,NaT
1240127,2017-05-09,2017-05-09 00:06:30,NaT
591319,2017-05-09,2017-05-09 00:08:21,2017-05-09 12:09:35
591318,2017-05-09,2017-05-09 00:08:21,2017-05-09 12:09:08
373946,2017-05-09,2017-05-09 00:10:30,NaT
590122,2017-05-09,2017-05-09 00:13:26,2017-05-09 12:13:35
243956,2017-05-09,2017-05-09 00:14:05,NaT
1240112,2017-05-09,2017-05-09 00:14:55,NaT
1029133,2017-05-09,2017-05-09 00:15:11,NaT


In [6]:
buste_v2['gps_datetime'] = pd.to_datetime(buste_v2['date'] + ' ' + buste_v2['timestamp'],format='%Y_%m_%d %H:%M:%S')
buste_v2['boarding_datetime'] = pd.to_datetime(buste_v2['date'] + ' ' + buste_v2['cardTimestamp'],format='%Y_%m_%d %H:%M:%S')
buste_v2['date'] = pd.to_datetime(buste_v2['date'], format='%Y_%m_%d')
buste_v2 = buste_v2.reset_index(drop=True)

In [30]:
print "Num unique boardings v2: {}".format(len(buste_v2.drop_duplicates(boarding_key_cols)))

Num unique boardings v2: 144669


In [10]:
buste_v2[['date','gps_datetime','boarding_datetime']].sort_values('gps_datetime')

,date,gps_datetime,boarding_datetime
175043,2017-05-09,2017-05-09 04:40:25,NaT
175045,2017-05-09,2017-05-09 04:41:19,2017-05-09 04:41:53
175044,2017-05-09,2017-05-09 04:41:19,2017-05-09 04:41:55
17330,2017-05-09,2017-05-09 04:41:21,NaT
175046,2017-05-09,2017-05-09 04:41:55,NaT
109096,2017-05-09,2017-05-09 04:42:01,NaT
341034,2017-05-09,2017-05-09 04:42:22,NaT
17331,2017-05-09,2017-05-09 04:42:36,NaT
280835,2017-05-09,2017-05-09 04:42:37,NaT
175047,2017-05-09,2017-05-09 04:42:45,NaT


#### Analyzing BUSTE single boarding data

In [11]:
other_cols = [col for col in buste_v1.columns if col not in first_cols]
cols_order = first_cols + other_cols

In [12]:
gps_by_boarding_v1 = buste_v1[cols_order] \
    .dropna(subset=boarding_key_cols) \
    .sort_values(boarding_key_cols)

num_single_boardings_bf_filter_v1 = len(gps_by_boarding_v1.drop_duplicates(subset=boarding_key_cols))
    
gps_by_boarding_v1['match_diff'] = (gps_by_boarding_v1['boarding_datetime'] - gps_by_boarding_v1['gps_datetime']).astype('timedelta64[s]')
gps_by_boarding_v1 = gps_by_boarding_v1.loc[gps_by_boarding_v1[np.abs(gps_by_boarding_v1['match_diff']) <= max_match_diff].groupby(['cardNum','boarding_datetime'])['match_diff'].idxmin()] \
                        .reset_index(drop=True)

print "Num single boardings before filtering: {}".format(num_single_boardings_bf_filter_v1)
print "Num single boardings after filtering: {}".format(len(gps_by_boarding_v1))

Num single boardings before filtering: 152706
Num single boardings after filtering: 144850


In [13]:
gps_by_boarding_v1.head(100)

,cardNum,boarding_datetime,gps_datetime,route,busCode,stopPointId,tripNum,shapeId,shapeSequence,shapeLat,...,gpsLon,distanceToShapePoint,timestamp,problem,birthdate,cardTimestamp,lineName,gender,date,match_diff
0,229948.0,2017-05-09 06:29:09,2017-05-09 06:28:51,654,HA017,35349,1.0,2106,6040781,-25.492611,...,-49.303553,20.905151,06:28:51,TRIP_PROBLEM,28/04/95,06:29:09,CAMPO ALEGRE,F,2017-05-09,18.0
1,257342.0,2017-05-09 07:48:58,2017-05-09 07:48:14,511,EA172,31195,22.0,2746,5846156,-25.498727,...,NaN,NaN,07:48:14,BETWEEN,24/01/71,07:48:58,SÃO FRANCISCO,F,2017-05-09,44.0
2,300327.0,2017-05-09 06:56:45,2017-05-09 06:55:54,654,HA240,36094,2.0,2106,6040871,-25.501307,...,-49.319971,0.737270,06:55:54,TRIP_PROBLEM,20/05/59,06:56:45,OP. CONTIGENCIA,F,2017-05-09,51.0
3,304127.0,2017-05-09 10:24:50,2017-05-09 10:23:32,175,BC010,31749,11.0,2743,5404834,-25.451146,...,-49.252913,39.855495,10:23:32,NO_PROBLEM,09/06/54,10:24:50,BOM RETIRO / PUC,M,2017-05-09,78.0
4,304127.0,2017-05-09 10:57:36,2017-05-09 10:57:08,370,LC016,25428,8.0,3669,6592082,-25.436333,...,NaN,NaN,10:57:08,BETWEEN,09/06/54,10:57:36,RUA XV / BARIGUI,M,2017-05-09,28.0
5,304627.0,2017-05-09 14:17:23,2017-05-09 14:14:45,370,BC032,30181,15.0,1887,5511364,-25.427767,...,-49.249085,8.345568,14:14:45,NO_PROBLEM,05/04/53,14:17:23,RUA XV / BARIGUI,F,2017-05-09,158.0
6,304627.0,2017-05-09 16:24:40,2017-05-09 16:05:27,370,LC020,30157,6.0,3669,6592013,-25.444290,...,-49.291258,5.801413,16:05:27,NO_PROBLEM,05/04/53,16:24:40,RUA XV / BARIGUI,F,2017-05-09,1153.0
7,306135.0,2017-05-09 07:56:49,2017-05-09 07:56:48,777,JC004,32097,6.0,2194,4299781,-25.446370,...,-49.275308,32.238018,07:56:48,TRIP_PROBLEM,29/10/52,07:56:49,V. VELHA,M,2017-05-09,1.0
8,306573.0,2017-05-09 07:50:41,2017-05-09 07:48:58,323,DA033,32543,18.0,1861,5583329,-25.450101,...,-49.206238,5.976702,07:48:58,NO_PROBLEM,03/04/59,07:50:41,MAD. CENTENÁRIO,F,2017-05-09,103.0
9,310241.0,2017-05-09 17:55:48,2017-05-09 17:55:39,468,DC090,31505,3.0,1926,4418075,-25.467945,...,NaN,NaN,17:55:39,BETWEEN,13/01/72,17:55:48,JD. ITIBERÊ,M,2017-05-09,9.0


In [14]:
gps_by_boarding_v2 = buste_v2[cols_order] \
    .dropna(subset=boarding_key_cols) \
    .sort_values(boarding_key_cols)
    
num_single_boardings_bf_filter_v2 = len(gps_by_boarding_v2.drop_duplicates(subset=boarding_key_cols))    
    
gps_by_boarding_v2['match_diff'] = (gps_by_boarding_v2['boarding_datetime'] - gps_by_boarding_v2['gps_datetime']).astype('timedelta64[s]')
gps_by_boarding_v2 = gps_by_boarding_v2.loc[gps_by_boarding_v2[np.abs(gps_by_boarding_v2['match_diff']) <= max_match_diff].groupby(['cardNum','boarding_datetime'])['match_diff'].idxmin()] \
                        .reset_index(drop=True)
    
print "Num single boardings before filtering: {}".format(num_single_boardings_bf_filter_v2)
print "Num single boardings after filtering: {}".format(len(gps_by_boarding_v2))

Num single boardings before filtering: 144668
Num single boardings after filtering: 139531


In [15]:
gps_by_boarding_v2.head(100)

,cardNum,boarding_datetime,gps_datetime,route,busCode,stopPointId,tripNum,shapeId,shapeSequence,shapeLat,...,gpsLon,distanceToShapePoint,timestamp,problem,birthdate,cardTimestamp,lineName,gender,date,match_diff
0,229948.0,2017-05-09 06:29:09,2017-05-09 06:28:49,654,HA017,35350,2.0,2953,3848651,-25.492612,...,NaN,NaN,06:28:49,BETWEEN,28/04/95,06:29:09,CAMPO ALEGRE,F,2017-05-09,20.0
1,257342.0,2017-05-09 07:48:58,2017-05-09 07:47:51,511,EA172,31195,4.0,2746,5846156,-25.498727,...,-49.247771,9.667548,07:47:51,NO_PROBLEM,24/01/71,07:48:58,SÃO FRANCISCO,F,2017-05-09,67.0
2,300327.0,2017-05-09 06:56:45,2017-05-09 06:55:54,654,HA240,36094,3.0,2106,6040871,-25.501307,...,-49.319971,0.737270,06:55:54,NO_PROBLEM,20/05/59,06:56:45,OP. CONTIGENCIA,F,2017-05-09,51.0
3,304127.0,2017-05-09 10:24:50,2017-05-09 10:22:28,175,BC010,31748,5.0,1743,5444496,-25.450869,...,-49.254015,13.270845,10:22:28,NO_PROBLEM,09/06/54,10:24:50,BOM RETIRO / PUC,M,2017-05-09,142.0
4,304127.0,2017-05-09 10:57:36,2017-05-09 10:42:18,370,LC016,3377,3.0,3669,6592154,-25.428220,...,-49.246896,5.426374,10:42:18,NO_PROBLEM,09/06/54,10:57:36,RUA XV / BARIGUI,M,2017-05-09,918.0
5,304627.0,2017-05-09 14:17:23,2017-05-09 14:16:09,370,BC032,3377,6.0,2789,5510110,-25.428214,...,-49.246675,18.124441,14:16:09,NO_PROBLEM,05/04/53,14:17:23,RUA XV / BARIGUI,F,2017-05-09,74.0
6,306135.0,2017-05-09 07:56:49,2017-05-09 07:56:48,777,JC004,32097,3.0,2194,4299781,-25.446370,...,-49.275308,32.238018,07:56:48,NO_PROBLEM,29/10/52,07:56:49,V. VELHA,M,2017-05-09,1.0
7,310241.0,2017-05-09 17:55:48,2017-05-09 17:55:42,468,DC090,30884,5.0,1926,4418132,-25.455683,...,-49.241156,8.323350,17:55:42,NO_PROBLEM,13/01/72,17:55:48,JD. ITIBERÊ,M,2017-05-09,6.0
8,312500.0,2017-05-09 13:26:11,2017-05-09 13:24:48,175,BC282,28632,6.0,2743,5404932,-25.431576,...,-49.271680,6.383659,13:24:48,NO_PROBLEM,26/02/82,13:26:11,BOM RETIRO / PUC,F,2017-05-09,83.0
9,312823.0,2017-05-09 13:35:49,2017-05-09 13:34:57,652,LA001,34977,13.0,2101,3834325,-25.534607,...,-49.331276,14.430964,13:34:57,NO_PROBLEM,03/09/88,13:35:49,V. VERDE,F,2017-05-09,52.0


#### Checking for differences between BUSTE v1 and BUSTE v2 in what refers to boarding data

#### Boarding records in BUSTE v2 but not in BUSTE v1

In [10]:
buste_v1[boarding_key_cols].join(buste_v2[boarding_key_cols], on=boarding_key_cols, how='inner')

ValueError: len(left_on) must equal the number of levels in the index of "right"

In [23]:
buste_v2_minus_v1.columns

Index([u'route', u'tripNum', u'shapeId', u'shapeSequence', u'shapeLat',
       u'shapeLon', u'distanceTraveledShape', u'busCode', u'gpsPointId',
       u'gpsLat', u'gpsLon', u'distanceToShapePoint', u'timestamp',
       u'stopPointId', u'problem', u'birthdate', u'cardTimestamp', u'lineName',
       u'cardNum', u'gender', u'date', u'gps_datetime', u'boarding_datetime'],
      dtype='object')

In [36]:
v2_v1_diff = pd.concat([buste_v1,buste_v2]).drop_duplicates(subset=boarding_key_cols)
print len(v2_v1_diff)

153997


In [33]:
v2_v1_diff

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,stopPointId,problem,birthdate,cardTimestamp,lineName,cardNum,gender,date,gps_datetime,boarding_datetime
5200,030,1.0,1716,6293813,-25.512778,-49.261923,24500.668,BR107,NaN,NaN,...,32606,BETWEEN,17/10/95,18:50:46,INTERBAIRROS III,3813513.0,F,2017-05-09,2017-05-09 18:26:46,2017-05-09 18:50:46
5201,030,1.0,1716,6293813,-25.512778,-49.261923,24500.668,BR107,NaN,NaN,...,32606,BETWEEN,13/05/82,19:07:19,INTERBAIRROS III,3585449.0,M,2017-05-09,2017-05-09 18:26:46,2017-05-09 19:07:19
5202,030,1.0,1716,6293813,-25.512778,-49.261923,24500.668,BR107,NaN,NaN,...,32606,BETWEEN,10/09/93,18:44:45,INTERBAIRROS III,3400407.0,M,2017-05-09,2017-05-09 18:26:46,2017-05-09 18:44:45
5203,030,1.0,1716,6293813,-25.512778,-49.261923,24500.668,BR107,NaN,NaN,...,32606,BETWEEN,18/10/83,18:54:25,INTERBAIRROS III,3676304.0,F,2017-05-09,2017-05-09 18:26:46,2017-05-09 18:54:25
5204,030,1.0,1716,6293650,-25.482830,-49.217635,17522.860,BR107,NaN,-25.482825,...,31909,NO_PROBLEM,31/12/68,19:13:01,INTERBAIRROS III,2980341.0,F,2017-05-09,2017-05-09 19:12:54,2017-05-09 19:13:01
10405,650,2.0,4166,6543261,-25.566751,-49.332993,1560.511,HA299,NaN,-25.566821,...,36294,NO_PROBLEM,24/06/90,05:48:13,OP. CONTIGENCIA,2106217.0,M,2017-05-09,2017-05-09 05:47:15,2017-05-09 05:48:13
10406,650,2.0,4166,6543261,-25.566751,-49.332993,1560.511,HA299,NaN,-25.566821,...,36294,NO_PROBLEM,20/08/55,05:48:23,OP. CONTIGENCIA,2258542.0,F,2017-05-09,2017-05-09 05:47:15,2017-05-09 05:48:23
10407,650,2.0,4166,6543261,-25.566751,-49.332993,1560.511,HA299,NaN,-25.566821,...,36294,NO_PROBLEM,19/11/91,05:48:18,OP. CONTIGENCIA,2758008.0,F,2017-05-09,2017-05-09 05:47:15,2017-05-09 05:48:18
10408,650,2.0,4166,6543261,-25.566751,-49.332993,1560.511,HA299,NaN,-25.566821,...,36294,NO_PROBLEM,19/11/91,05:48:10,OP. CONTIGENCIA,2758008.0,F,2017-05-09,2017-05-09 05:47:15,2017-05-09 05:48:10
10409,650,2.0,4166,6543269,-25.565282,-49.333792,1742.133,HA299,NaN,-25.565250,...,36299,NO_PROBLEM,14/12/64,05:48:44,OP. CONTIGENCIA,2236624.0,M,2017-05-09,2017-05-09 05:48:28,2017-05-09 05:48:44


In [38]:
buste_v2_minus_v1 = pd.concat([buste_v1,buste_v1,buste_v2]).drop_duplicates(subset=boarding_key_cols)
print len(buste_v2_minus_v1)

153997


In [35]:
buste_v2_minus_v1

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,stopPointId,problem,birthdate,cardTimestamp,lineName,cardNum,gender,date,gps_datetime,boarding_datetime
493,372,8.0,1890,6137161,-25.424189,-49.203354,7978.667,CC170,NaN,-25.424156,...,30791,NO_PROBLEM,02/01/97,19:29:12,TARUMÃ,3407454.0,F,2017-05-09,2017-05-09 19:28:55,2017-05-09 19:29:12
498,372,8.0,1890,6137205,-25.415692,-49.202210,9268.598,CC170,NaN,-25.415675,...,30769,NO_PROBLEM,07/01/79,19:33:21,TARUMÃ,3286745.0,F,2017-05-09,2017-05-09 19:32:34,2017-05-09 19:33:21
3550,860,5.0,2240,4494847,-25.444550,-49.288534,2040.211,LC019,NaN,-25.444673,...,44544,NO_PROBLEM,29/02/00,12:01:00,V. SANDRA,3809621.0,F,2017-05-09,2017-05-09 12:00:39,2017-05-09 12:01:00
3551,860,5.0,2240,4494847,-25.444550,-49.288534,2040.211,LC019,NaN,-25.444673,...,44544,NO_PROBLEM,16/08/01,12:01:15,V. SANDRA,3183242.0,M,2017-05-09,2017-05-09 12:00:39,2017-05-09 12:01:15
3552,860,5.0,2240,4494847,-25.444550,-49.288534,2040.211,LC019,NaN,-25.444673,...,44544,NO_PROBLEM,15/05/61,12:01:07,V. SANDRA,3745967.0,F,2017-05-09,2017-05-09 12:00:39,2017-05-09 12:01:07
3553,860,5.0,2240,4494847,-25.444550,-49.288534,2040.211,LC019,NaN,-25.444673,...,44544,NO_PROBLEM,27/02/67,12:01:25,V. SANDRA,3727160.0,F,2017-05-09,2017-05-09 12:00:39,2017-05-09 12:01:25
3554,860,5.0,2240,4494852,-25.445697,-49.291446,2359.531,LC019,NaN,-25.445708,...,40642,NO_PROBLEM,13/07/94,12:03:15,V. SANDRA,3214403.0,M,2017-05-09,2017-05-09 12:01:47,2017-05-09 12:03:15
3555,860,5.0,2240,4494852,-25.445697,-49.291446,2359.531,LC019,NaN,-25.445708,...,40642,NO_PROBLEM,13/07/94,12:03:18,V. SANDRA,3214403.0,M,2017-05-09,2017-05-09 12:01:47,2017-05-09 12:03:18
3556,860,5.0,2240,4494852,-25.445697,-49.291446,2359.531,LC019,NaN,-25.445708,...,40642,NO_PROBLEM,13/09/96,12:02:52,V. SANDRA,3816062.0,F,2017-05-09,2017-05-09 12:01:47,2017-05-09 12:02:52
3557,860,5.0,2240,4494852,-25.445697,-49.291446,2359.531,LC019,NaN,-25.445708,...,40642,NO_PROBLEM,16/05/02,12:03:09,V. SANDRA,3806683.0,M,2017-05-09,2017-05-09 12:01:47,2017-05-09 12:03:09


In [23]:
buste_v2_minus_v1.columns

Index([u'route', u'tripNum', u'shapeId', u'shapeSequence', u'shapeLat',
       u'shapeLon', u'distanceTraveledShape', u'busCode', u'gpsPointId',
       u'gpsLat', u'gpsLon', u'distanceToShapePoint', u'timestamp',
       u'stopPointId', u'problem', u'birthdate', u'cardTimestamp', u'lineName',
       u'cardNum', u'gender', u'date', u'gps_datetime', u'boarding_datetime'],
      dtype='object')

In [39]:
buste_v1_minus_v2 = pd.concat([buste_v2,buste_v2,buste_v1]).drop_duplicates(subset=boarding_key_cols)
print len(buste_v1_minus_v2)

153997


In [21]:
buste_v1_minus_v2

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,stopPointId,problem,birthdate,cardTimestamp,lineName,cardNum,gender,date,gps_datetime,boarding_datetime
5200,030,1.0,1716,6293813,-25.512778,-49.261923,24500.668,BR107,NaN,NaN,...,32606,BETWEEN,17/10/95,18:50:46,INTERBAIRROS III,3813513.0,F,2017-05-09,2017-05-09 18:26:46,2017-05-09 18:50:46
5201,030,1.0,1716,6293813,-25.512778,-49.261923,24500.668,BR107,NaN,NaN,...,32606,BETWEEN,13/05/82,19:07:19,INTERBAIRROS III,3585449.0,M,2017-05-09,2017-05-09 18:26:46,2017-05-09 19:07:19
5202,030,1.0,1716,6293813,-25.512778,-49.261923,24500.668,BR107,NaN,NaN,...,32606,BETWEEN,10/09/93,18:44:45,INTERBAIRROS III,3400407.0,M,2017-05-09,2017-05-09 18:26:46,2017-05-09 18:44:45
5203,030,1.0,1716,6293813,-25.512778,-49.261923,24500.668,BR107,NaN,NaN,...,32606,BETWEEN,18/10/83,18:54:25,INTERBAIRROS III,3676304.0,F,2017-05-09,2017-05-09 18:26:46,2017-05-09 18:54:25
5204,030,1.0,1716,6293650,-25.482830,-49.217635,17522.860,BR107,NaN,-25.482825,...,31909,NO_PROBLEM,31/12/68,19:13:01,INTERBAIRROS III,2980341.0,F,2017-05-09,2017-05-09 19:12:54,2017-05-09 19:13:01
10405,650,2.0,4166,6543261,-25.566751,-49.332993,1560.511,HA299,NaN,-25.566821,...,36294,NO_PROBLEM,24/06/90,05:48:13,OP. CONTIGENCIA,2106217.0,M,2017-05-09,2017-05-09 05:47:15,2017-05-09 05:48:13
10406,650,2.0,4166,6543261,-25.566751,-49.332993,1560.511,HA299,NaN,-25.566821,...,36294,NO_PROBLEM,20/08/55,05:48:23,OP. CONTIGENCIA,2258542.0,F,2017-05-09,2017-05-09 05:47:15,2017-05-09 05:48:23
10407,650,2.0,4166,6543261,-25.566751,-49.332993,1560.511,HA299,NaN,-25.566821,...,36294,NO_PROBLEM,19/11/91,05:48:18,OP. CONTIGENCIA,2758008.0,F,2017-05-09,2017-05-09 05:47:15,2017-05-09 05:48:18
10408,650,2.0,4166,6543261,-25.566751,-49.332993,1560.511,HA299,NaN,-25.566821,...,36294,NO_PROBLEM,19/11/91,05:48:10,OP. CONTIGENCIA,2758008.0,F,2017-05-09,2017-05-09 05:47:15,2017-05-09 05:48:10
10409,650,2.0,4166,6543269,-25.565282,-49.333792,1742.133,HA299,NaN,-25.565250,...,36299,NO_PROBLEM,14/12/64,05:48:44,OP. CONTIGENCIA,2236624.0,M,2017-05-09,2017-05-09 05:48:28,2017-05-09 05:48:44


In [22]:
boarding_key_cols

['cardNum', 'boarding_datetime']